In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('serious-injury-outcome-indicators-2000-2020-CSV.csv')

In [3]:
df.head()

Series_reference   Period            Type  Data_value   Lower_CI   Upper_CI  \
0            W_A11  2000-02  Moving average   59.666667  50.925823  68.407510   
1            W_A11  2001-03  Moving average   60.000000  51.234775  68.765225   
2            W_A11  2002-04  Moving average   59.000000  50.308125  67.691875   
3            W_A11  2003-05  Moving average   59.000000  50.308125  67.691875   
4            W_A11  2004-06  Moving average   61.333333  52.471252  70.195415   

      Units Indicator    Cause Validation Population       Age Severity  
0  Injuries    Number  Assault  Validated  Whole pop  All ages    Fatal  
1  Injuries    Number  Assault  Validated  Whole pop  All ages    Fatal  
2  Injuries    Number  Assault  Validated  Whole pop  All ages    Fatal  
3  Injuries    Number  Assault  Validated  Whole pop  All ages    Fatal  
4  Injuries    Number  Assault  Validated  Whole pop  All ages    Fatal

In [4]:
df.isnull().sum()

Series_reference    0
Period              0
Type                0
Data_value          0
Lower_CI            0
Upper_CI            0
Units               0
Indicator           0
Cause               0
Validation          0
Population          0
Age                 0
Severity            0
dtype: int64

In [5]:
df.shape

(2748, 13)

In [6]:
df['Type'].unique()

array(['Moving average', 'Single year'], dtype=object)

In [7]:
df['Units'].unique()

array(['Injuries', 'Per 100,000 people', 'Per 100,000 FTEs',
       'Per billion km', 'Per thousand registered vehicles'], dtype=object)

In [8]:
df['Indicator'].unique()

array(['Number', 'Age-standardised rate', 'Rate per billion km',
       'Rate per thousand registered vehicles'], dtype=object)

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df['Type']=le.fit_transform(df['Type'])
df['Units']=le.fit_transform(df['Units'])
df['Indicator']=le.fit_transform(df['Indicator'])
df['Cause']=le.fit_transform(df['Cause'])
df['Validation']=le.fit_transform(df['Validation'])
df['Population']=le.fit_transform(df['Population'])
df['Age']=le.fit_transform(df['Age'])
df['Severity']=le.fit_transform(df['Severity'])

In [10]:
x = df.iloc[:,2:12]
x

Type  Data_value   Lower_CI   Upper_CI  Units  Indicator  Cause  \
0        0   59.666667  50.925823  68.407510      0          1      1   
1        0   60.000000  51.234775  68.765225      0          1      1   
2        0   59.000000  50.308125  67.691875      0          1      1   
3        0   59.000000  50.308125  67.691875      0          1      1   
4        0   61.333333  52.471252  70.195415      0          1      1   
...    ...         ...        ...        ...    ...        ...    ...   
2743     1   46.000000  32.706878  59.293122      0          1      8   
2744     1   41.000000  28.450107  53.549893      0          1      8   
2745     1   36.000000  24.240216  47.759784      0          1      8   
2746     1   40.000000  27.604099  52.395901      0          1      8   
2747     1   41.000000  28.450107  53.549893      0          1      8   

      Validation  Population  Age  
0              1           2    3  
1              1           2    3  
2              1           2    3  
3              1           2    3  
4              1           2    3  
...          ...         ...  ...  
2743           1           0    0  
2744           1           0    0  
2745           1           0    0  
2746           1           0    0  
2747           1           0    0  

[2748 rows x 10 columns]

In [11]:
y = df.iloc[:,-1:]
y

Severity
0            0
1            0
2            0
3            0
4            0
...        ...
2743         1
2744         1
2745         1
2746         1
2747         1

[2748 rows x 1 columns]

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train,X_test,y_train,y_test = train_test_split(x,y,random_state=0,test_size=0.2)

In [14]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

features_new = SelectKBest(score_func=chi2,k=5)
feature = features_new.fit(x,y)

In [15]:
dfscores = pd.DataFrame(feature.scores_,columns=['Score'])
dfcolumns = pd.DataFrame(x.columns)

In [16]:
feature_rank = pd.concat([dfcolumns,dfscores],axis=1)

In [17]:
feature_rank.columns = ['Features','Scores']

In [18]:
feature_rank.nlargest(5,'Scores')

Features         Scores
3    Upper_CI  361352.509286
1  Data_value  353866.064143
2    Lower_CI  346560.133835
9         Age     188.166603
8  Population     180.238996

In [19]:
from keras.layers import Dense
from keras.models import Sequential

In [20]:
model = Sequential()

In [32]:
model.add(Dense(units=20,input_dim=5,activation='relu'))
model.add(Dense(units=5,activation='relu'))
model.add(Dense(units=10,activation='relu'))
model.add(Dense(units=1,activation='softmax'))

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                120       
                                                                 
 dense_1 (Dense)             (None, 20)                420       
                                                                 
 dense_2 (Dense)             (None, 20)                420       
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
 dense_4 (Dense)             (None, 20)                40        
                                                                 
 dense_5 (Dense)             (None, 5)                 105       
                                                                 
 dense_6 (Dense)             (None, 20)                1

In [34]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [35]:
x = df[['Data_value','Lower_CI','Upper_CI','Age','Population']]
x

Data_value   Lower_CI   Upper_CI  Age  Population
0      59.666667  50.925823  68.407510    3           2
1      60.000000  51.234775  68.765225    3           2
2      59.000000  50.308125  67.691875    3           2
3      59.000000  50.308125  67.691875    3           2
4      61.333333  52.471252  70.195415    3           2
...          ...        ...        ...  ...         ...
2743   46.000000  32.706878  59.293122    0           0
2744   41.000000  28.450107  53.549893    0           0
2745   36.000000  24.240216  47.759784    0           0
2746   40.000000  27.604099  52.395901    0           0
2747   41.000000  28.450107  53.549893    0           0

[2748 rows x 5 columns]

In [36]:
y

Severity
0            0
1            0
2            0
3            0
4            0
...        ...
2743         1
2744         1
2745         1
2746         1
2747         1

[2748 rows x 1 columns]

In [37]:
model.fit(x,y,epochs=800)

Epoch 1/800
86/86 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 2/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 3/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 4/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 5/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 6/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 7/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 8/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 9/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 10/800
86/86 [==============================] - 0s 2ms/step - loss:

86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 80/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 81/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 82/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 83/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 84/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 85/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 86/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 87/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 88/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0

86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 158/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 159/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 160/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 161/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 162/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 163/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 164/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 165/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 166/800
86/86 [==============================] - 0s 2ms/step - 

86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 236/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 237/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 238/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 239/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 240/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 241/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 242/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 243/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 244/800
86/86 [==============================] - 0s 2ms/step - 

86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 314/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 315/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 316/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 317/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 318/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 319/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 320/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 321/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 322/800
86/86 [==============================] - 0s 2ms/step - 

...

86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 470/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 471/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 472/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 473/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 474/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 475/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 476/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 477/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 478/800
86/86 [==============================] - 0s 2ms/step - 

86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 548/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 549/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 550/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 551/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 552/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 553/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 554/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 555/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 556/800
86/86 [==============================] - 0s 2ms/step - 

86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 626/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 627/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 628/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 629/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 630/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 631/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 632/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 633/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 634/800
86/86 [==============================] - 0s 2ms/step - 

86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 704/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 705/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 706/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 707/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 708/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 709/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 710/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 711/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 712/800
86/86 [==============================] - 0s 2ms/step - 

86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 782/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 783/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 784/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 785/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 786/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 787/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 788/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 789/800
86/86 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3661
Epoch 790/800
86/86 [==============================] - 0s 2ms/step - 